In [78]:

# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [79]:
data = pd.read_csv("/kaggle/input/food-delivery-time/deliverytime.csv")

In [80]:
data.sample(5)

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Type_of_order,Type_of_vehicle,Time_taken(min)
31325,BF4F,KOLRES15DEL02,29,4.6,22.552672,88.352885,22.572672,88.372885,Drinks,motorcycle,22
8918,E075,ALHRES16DEL03,27,5.0,25.450377,81.834236,25.510377,81.894236,Buffet,motorcycle,24
31899,AC1B,RANCHIRES19DEL01,29,4.6,23.333017,85.317200,23.373017,85.357200,Buffet,scooter,22
11752,9AAD,CHENRES08DEL03,39,4.1,13.022394,80.242439,13.052394,80.272439,Drinks,scooter,33
22750,4E6C,COIMBRES05DEL01,34,4.7,11.022169,76.999594,11.062169,77.039594,Drinks,motorcycle,28


In [81]:
print("<**********************************Decsribe**********************************>")
print("")
print(data.describe())
print("")
print("<**********************************Info**********************************>")
print("")
print(data.info())
print("")
print("<**********************************Columns**********************************>")
print("")
print(data.columns)
print("<**********************************Null**********************************>")
print("")
print(data.isnull().sum())

<**********************************Decsribe**********************************>

       Delivery_person_Age  Delivery_person_Ratings  Restaurant_latitude  \
count         45593.000000             45593.000000         45593.000000   
mean             29.544075                 4.632367            17.017729   
std               5.696793                 0.327708             8.185109   
min              15.000000                 1.000000           -30.905562   
25%              25.000000                 4.600000            12.933284   
50%              29.000000                 4.700000            18.546947   
75%              34.000000                 4.800000            22.728163   
max              50.000000                 6.000000            30.914057   

       Restaurant_longitude  Delivery_location_latitude  \
count          45593.000000                45593.000000   
mean              70.231332                   17.465186   
std               22.883647                    7.335122   

In [82]:
data.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Type_of_order,Type_of_vehicle,Time_taken(min)
0,4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,Snack,motorcycle,24
1,B379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,Snack,scooter,33
2,5D6D,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,Drinks,motorcycle,26
3,7A6A,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,Buffet,motorcycle,21
4,70A2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,Snack,scooter,30


In [83]:
data.value_counts("Type_of_order")

Type_of_order
Snack      11533
Meal       11458
Drinks     11322
Buffet     11280
Name: count, dtype: int64

In [84]:
data["Type_of_vehicle"] = LabelEncoder().fit_transform(data["Type_of_vehicle"])
data["Type_of_order"] = LabelEncoder().fit_transform(data["Type_of_order"])

# Data'nın içersinde bulunan kordinatları kullanarak uzaklığı hesaplayacağız

In [85]:
def haversine_distance(row):
    R = 6371  # Dünya'nın yarıçapı (km)

    # Enlem ve boylam farklarını radyan cinsinden hesaplama
    delta_lat = math.radians(row['Delivery_location_latitude'] - row['Restaurant_latitude'])
    delta_lon = math.radians(row['Delivery_location_longitude'] - row['Restaurant_longitude'])

    # Haversine formülüne göre mesafeyi hesaplama
    a = math.sin(delta_lat/2)**2 + math.cos(math.radians(row['Restaurant_latitude'])) * math.cos(math.radians(row['Delivery_location_latitude'])) * math.sin(delta_lon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = R * c

    return distance

data['Delivery_Distance_km'] = data.apply(haversine_distance, axis=1)
data.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Type_of_order,Type_of_vehicle,Time_taken(min),Delivery_Distance_km
0,4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,3,2,24,3.025149
1,B379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,3,3,33,20.183530
2,5D6D,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,1,2,26,1.552758
3,7A6A,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,0,2,21,7.790401
4,70A2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,3,3,30,6.210138


### Şimdi de oluşturduğumuz stünü kullanarak kuryenin saatlik hızını hesaplayacağız bunu yaparken teslim zamanını 60'a bölüp sonra kurye km uzaklığına bölüp saatlik hızına ulaşacağız 

In [86]:
data['Speed_km_per_hr'] = data['Delivery_Distance_km'] / (data['Time_taken(min)'] / 60)

In [87]:
data.columns

Index(['ID', 'Delivery_person_ID', 'Delivery_person_Age',
       'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Type_of_order', 'Type_of_vehicle',
       'Time_taken(min)', 'Delivery_Distance_km', 'Speed_km_per_hr'],
      dtype='object')

In [89]:
x=data[['Delivery_person_Age', 'Delivery_person_Ratings', 'Speed_km_per_hr',"Type_of_order","Type_of_vehicle",'Delivery_Distance_km']]
y=data['Time_taken(min)']

In [92]:
# Splitting the data
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

In [94]:
# Model training
model = GradientBoostingRegressor(random_state=42)
model.fit(x_train, y_train)

GradientBoostingRegressor(random_state=42)

In [96]:
# Model evaluation
y_pred = model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'MSE: {mse}, R2: {r2}')

MSE: 2.5902123198650484, R2: 0.9704577170945483
